#### Supervised classification with random forest using GEE

In [ ]:
# Import and initilize ee library
import ee, json, folium
ee.Initialize()

In [44]:
# Import boundary and labeled regions

with open('kings_creek.geojson') as file:
    kc_json = json.load(file)

with open('forest.geojson') as file:
    forest_json = json.load(file)

with open('grassland.geojson') as file:
    grassland_json = json.load(file)
 


In [45]:
# Define the ee.Geometry

kc_geometry = ee.Geometry(kc_json['features'][0]['geometry'])

forest_geometry = ee.Feature(forest_json['features'][0]['geometry'])
grassland_geometry = ee.Feature(grassland_json['features'][0]['geometry'])

# Add classes to features
forest_geometry = forest_geometry.set('land_cover', 0)
grassland_geometry = grassland_geometry.set('land_cover', 1)

# Merge train dataset into a collection
train_regions=ee.FeatureCollection([forest_geometry, grassland_geometry])

In [46]:
# Define source dataset

dataset = ee.Image('COPERNICUS/S2_SR/20220723T170901_20220723T172122_T14SQJ').clip(kc_geometry)


In [47]:
# Generate training data

train_data = dataset.sampleRegions(collection = train_regions, properties=['land_cover'], scale=30)

In [48]:
# Train Random forest algorithm

classifier = ee.Classifier.smileRandomForest(10).train(features = train_data,
                                                       classProperty='land_cover', \
                                                       inputProperties=dataset.bandNames() \
                                                      )

In [49]:
# Classify the entire King's Creek Watershed
regions = dataset.classify(classifier).reduceToVectors(scale = 30, geometry = kc_geometry).getInfo()

In [50]:
# Create Folium Map
m = folium.Map(location=[39.07648, -96.592], zoom_start=12)

# Get Color for different polygons
def get_polygon_color(polygon):
    colors = ['#7b3294', '#008837']
    color = colors[polygon['properties']['label']]
    return {'fillColor': color, 'fillOpacity': 0.8}
    
    
cluster_layer=folium.GeoJson(regions, name="Konza Praririe Clustered Regions", style_function = get_polygon_color)
cluster_layer.add_to(m)

m